In [2]:
import pandas as pd
import requests
import json
import datetime
import time
import random

In [3]:
import requests
import requests.auth

client_id = "42096"
client_secret = "97080792c6d30a4178965e41f1ca15de"
TOKEN_URL = "https://www.deviantart.com/oauth2/token"
REDIRECT_URI = "https://www.deviantart.com/oauth2/authorize"

def get_token():
    client_auth = requests.auth.HTTPBasicAuth(client_id, client_secret)
    post_data = {"grant_type": "client_credentials",
                 "redirect_uri": REDIRECT_URI}
    response = requests.post(TOKEN_URL,
                             auth=client_auth,
                             data=post_data)
    token_json = response.json()
    return token_json["access_token"]

In [6]:
def get_a_week(numdays):
    date_list=[(datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range(0, numdays)]
    return date_list

In [3]:
def get_dailydeviations_info(token, day):
    api_url = f"https://www.deviantart.com/api/v1/oauth2/browse/dailydeviations?access_token={token}"
    response = requests.get(api_url, params={'date': day})
    return response

In [8]:
token = get_token()

In [9]:
get_dailydeviations_info(token,datetime.date.today())

<Response [200]>

In [5]:
def get_response_rate(response):    
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    elif response.status_code == 404:
        return 'user_done'
    elif response.status_code == 429:
        return 'too_many_requests'
    elif response.status_code == 500:
        return 'server error'
    elif response.status_code == 401:
        return 'get new token'

In [6]:
date=random.choice(get_a_week(7))

In [11]:
def get_seed(numdays):
    token=get_token()
    date=random.choice(get_a_week(numdays))
    response=get_dailydeviations_info(token, date)
    j=get_response_rate(response)
    if j=='too_many_requests':
        time.sleep(60)
        j=get_response_rate(response)
    elif j=='get new token': 
        token=get_token()
        j=get_response_rate(response)
    return random.choice([i['author'] for i in j['results']])

In [1]:
def get_friends(username, page):
    token=get_token()
    api_url=f"https://www.deviantart.com/api/v1/oauth2/user/friends/{username}?access_token={token}"
    response = requests.get(api_url, params={'offset': page, 'limit': 50})
    return response

In [5]:
def parse_friends(friends):
    users=pd.DataFrame()
    #print(friends.keys())
    has_more=friends.get('has_more')
    #next_offset=friends['next_offset']
    for i in friends['results']:
        a={ 'username': i['user']['username'],
            'is_watching': i['is_watching'],
            'watches_you': i['watches_you'],
            'last_visit': i['lastvisit'],
            'activity': i['watch']['activity'],
            'collections': i['watch']['collections'],
            'critiques':  i['watch']['critiques'],
            'deviations': i['watch']['deviations'],
            'forum_threads': i['watch']['forum_threads'],
            'friend': i['watch']['friend'],
            'journals': i['watch']['journals'],
            'scraps': i['watch']['scraps']}
        users=users.append(pd.DataFrame.from_dict(a, orient='index').T)
    return (has_more, a)

In [10]:
def get_all_freinds(user):
    token=get_token()
    all_friends=pd.DataFrame()
    has_more=True
    for i in range(0,500):
        if has_more==True:
            resp=get_friends(user, i)
            friends=get_response_rate(resp)
            #print(friends)
            if friends=='too_many_requests':
                time.sleep(60)
                friends=get_response_rate(resp)
            if friends != None:
                has_more, users=parse_friends(friends)
            else:
                return 'no friends'
            if len(users)>0:
                all_friends=all_friends.append(users)
            else:
                return 'no friends'
    all_friends.to_csv(path+user+'_freinds.csv')
    print('freinds printed')
    return all_friends.sample(1)['username'].values[0]

In [11]:
def get_user_profile(username):
    token=get_token()
    #url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
    url=f"https://www.deviantart.com/api/v1/oauth2/user/profile/{username}?access_token={token}"
    response = requests.get(url)
    return response

In [12]:
def parse_user_profile(user):
    s=get_user_profile(user)
    d=json.loads(s.text)
    if 'error' not in d.keys():
        a={'user':user,
        'real_name': d.get('real_name'),
        'profil_url': d['profile_url'],
        'tags': d['tagline'],
        'country': d['countryid'],
        'bio': d['bio'],
        'level': d['artist_level'],
        'specialty' :d['artist_specialty']}
        a.update(d['stats'])
        return a
    else:
        return 'error'

In [19]:
res=pd.DataFrame()

In [20]:
token=get_token()
path='/Users/orsi/Dropbox/devianrt/friends/'
path_results='/Users/orsi/Dropbox/devianrt/results/'
date=random.choice(get_a_week(7))
response=get_dailydeviations_info(client_secret, date)
user=get_seed(6)['username']
#res=pd.DataFrame.from_dict(parse_user_profile(user, token), 'index').T
for i in range(0,500):
    if i>1:
        time.sleep(100)
    else:
        try:
            if i%100==0 and i>1:
                res.to_csv(path_results+'users'+str(i)+'.csv')
                user=get_seed(6)['username']
            else:
                user=get_all_freinds(user)
                if user == 'no friends':
                    user = get_seed(6)['username']
            k=parse_user_profile(user)
            if k!= 'error':
                res=res.append(pd.DataFrame.from_dict(k, 'index').T)
            else:
                print('wtf')
                pass
        except:
            try:
                print('hello')
                token=get_token()
                user=get_seed(14)['username']
                user=get_all_freinds(user)
                response=get_dailydeviations_info(client_secret, date)
                res=res.append(pd.DataFrame.from_dict(parse_user_profile(user), 'index').T)
            except:
                print(len(res))
                pass

hello
0
hello
0


KeyboardInterrupt: 

In [21]:
res

""


In [127]:
res.to_csv('new_users'+len(res)'.csv')

In [ ]:
get_all_freinds(user, token)

In [ ]:
all_friends=pd.DataFrame()
has_more=True
for i in range(0,500):
    if has_more==True:
        resp=get_friends(user, token, i)
        friends=get_response_rate(resp)
        has_more, users=parse_friends(friends)
        all_friends=all_friends.append(users)
#all_friends.to_csv(path+user+'_freinds.csv')

In [ ]:
get_response_rate(resp)

In [ ]:
has_more, users=parse_friends(friends)

In [ ]:
has_more